# 1. Scraping

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Load the postal code table in the Wikipedia page.  
*Assumption:*
- *the postal code table is the first table in the page*
- *order of columns of the table is: Postalcode, Borough, and Neighbourhood*

In [31]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_p = dfs[0]
df_p.head(2)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned


### Fixup table
*Assumption*
- *Missing value is represented by keyword* **Not assigned**

In [59]:
# Fix column names
df_p.columns = ['PostalCode', 'Borough', 'Neighborhood']

# Make 'Not assigned' NaN
df_p.replace({'Not assigned': pd.np.nan}, inplace=True)

# Fixup Neighborhood with Borough
df_p.fillna(value={'Neighborhood': df_p['Borough']}, inplace=True)

# Drop NaN
df_p.dropna(subset=['Borough'], inplace=True)

# Group Neighborhood by Zip code
df_pa = df_p.groupby('PostalCode').agg({'Borough': lambda x: x.iloc[0], 'Neighborhood': lambda x: ', '.join(x)})

# Reset index
df_pa.reset_index(inplace=True)

df_pa.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Print shape

In [58]:
df_pa.shape

(103, 3)